### Energy Efficiency Indices for carbon foot printing and as instrument for CO2 emission reduction of inland vessels
Federal Ministry of Transport and Digital Infrastructure, Germany; DST; CESNI 

----------------------------

### The inland EEDI approach includes the attained EEDI and required EEDI for inland waterway navigation, covering 4 representative vessel types and 3 waterway conditions: 
#### four representative vessel types
- dry cargo and container selfpropelled ships
- tankers
- pushed convoys
- passenger ships

#### three waterway conditions: 
- deep calm water (h>7.5m), 
- shallow water with currents (3.5 m <= h <= 7.5 m), 
- cannal: major canals with trapezoidal cross-section, e.g. the MD canal.

#### references
- https://www.cesni.eu/wp-content/uploads/2021/03/cesnipt_energyindex_de.pdf
- https://platina3.eu/download/gernot-paulii-and-jens-ley-on-energy-efficiency-indices-as-an-instrument-for-the-reduction-of-co2-emissions-of-inland-vessels/
- https://link.springer.com/book/10.1007/978-3-030-77325-0 chapter 6.2.4

In [1]:
import math
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import plotly.express as px
from plotly.subplots import make_subplots

### attained EEDI   (g CO2 / ton-km)
for different vessel types and sailing conditions, use the same EEDI_attained formula with different P_D and V_g  to get the attained EEDI. 

Note that the P_D calculations for pushed convoys and self-propelled ships are the same in shallow water!! 
then the use the same formula for EEDI_attained, which means in shallow water the method has no difference among vessel types for getting EEDI_attained


In [2]:
def calculate_attained_EEDI( B, h, vessel_type, DWT, V_g):
    '''
    calculate_attained_EEDI for inland vessels.
    vessel types include 1) dry cargo and container selfpropelled ships, 2) tankers, 3) pushed convoys and 4) passenger ships.
   
    Note 
    - the calculation covers waterway conditions of deep calm water and shallow current water, vessel types of dry cargo and container selfpropelled ship, and pushed convoy so far, 
    for the RhineARA case study. Other water conditions and vessel types can be added into calculation when needed.
    
    input
    - B: ship beam (m)
    - h:  water depth, deep water range h >7.5 m, shallow water 3.5 m <= h <= 7.5 m;  lower than 3.5 m is excluded in the approach
    - vessel_type: "dry cargo", "container", "pushed convoy"   
    - DWT: capacity, [ton]
    - V_g: Velocity over ground, [km/h]
    - CF: CO2 Factor of gasoil, [g CO2/ g diesel] 
    - SFC: Specific fuel consumption, [g diesel/kWh] 
    
    '''
    def get_P_D( B, h, vessel_type, DWT):
        ''' P_D: Applied delivered power depending on ship type and sailing direction up-downstream
        '''
        if h > 7.5: 
            if vessel_type == "dry cargo" or vessel_type == "container":
                P_D = 0.262 * DWT        
            elif vessel_type == "pushed convoy":
                P_D = (0.146 + 0.25 * math.exp( B / (-11))) * DWT


        elif 3.5 <= h <= 7.5:
            if vessel_type == "dry cargo" or vessel_type == "container":
                P_D = (0.375 + 0.0625 * math.exp(-0.13 * B) - 0.5 * math.exp( h / (-2.8))) * DWT        
            elif vessel_type == "pushed convoy":
                P_D = (0.375 + 0.0625 * math.exp(-0.13 * B) - 0.5 * math.exp( h / (-2.8))) * DWT  
                # note that the P_D calculations for pushed convoys and self-propelled ships are the same in shallow water!! 
                # then the use the same formula for EEDI_attained, which means in shallow water the method has no difference among vessel types for getting EEDI_attained

        else:
            print ("out of the water depth range or wrong vessel type input")        

        return P_D
    
    
    CF = 3.206 
    SFC = 220 
    
    EEDI_attained = CF * SFC * get_P_D(B, h, vessel_type, DWT) / (DWT * V_g)
    
    return EEDI_attained

In [3]:
def get_P_D( B, h, vessel_type, DWT):
        ''' P_D: Applied delivered power depending on ship type and sailing direction up-downstream
        '''
        if h > 7.5: 
            if vessel_type == "dry cargo" or vessel_type == "container":
                P_D = 0.262 * DWT        
            elif vessel_type == "pushed convoy":
                P_D = (0.146 + 0.25 * math.exp( B / (-11))) * DWT


        elif 3.5 <= h <= 7.5:
            if vessel_type == "dry cargo" or vessel_type == "container":
                P_D = (0.375 + 0.0625 * math.exp(-0.13 * B) - 0.5 * math.exp( h / (-2.8))) * DWT        
            elif vessel_type == "pushed convoy":
                P_D = (0.375 + 0.0625 * math.exp(-0.13 * B) - 0.5 * math.exp( h / (-2.8))) * DWT  
                # note that the P_D calculations for pushed convoys and self-propelled ships are the same in shallow water!! 
                # then the use the same formula for EEDI_attained, which means in shallow water the method has no difference among vessel types for getting EEDI_attained

        else:
            print ("out of the water depth range or wrong vessel type input")        

        return P_D

In [4]:
EEDI_attained = calculate_attained_EEDI( B = 10, h =4, vessel_type = "container", DWT =3000, V_g=11)
EEDI_attained

17.45395893226047

In [5]:
EEDI_attained = calculate_attained_EEDI( B = 10, h =10, vessel_type = "container", DWT =3000, V_g=11)
EEDI_attained

16.799439999999997

In [6]:
EEDI_attained = calculate_attained_EEDI( B = 20, h =10, vessel_type = "pushed convoy", DWT =18000, V_g=13)
EEDI_attained

10.12297795151502

In [7]:
EEDI_attained = calculate_attained_EEDI( B = 10, h =10, vessel_type = "pushed convoy", DWT =3000, V_g=11)
EEDI_attained

15.819851854111999

### required EEDI (g CO2 / ton-km)
for different vessel types and sailing conditions, use different EEDI_req formula below to get the required EEDI for each situation

Note that the required_EEDI calculations for pushed convoys and self-propelled ships are the same in deep calm water!!

In [8]:
def calculate_required_EEDI(h, U_c, vessel_type, DWT):
    ''' the calculation covers waterway conditions of deep calm water and shallow current water, vessel types of dry cargo and container selfpropelled ship, and pushed convoy so far, 
    for the RhineARA case study. Other water conditions and vessel types can be added into calculation when needed.
    
    - U_c: current speed, [km/h]
    '''
    if h > 7.5: 
        if vessel_type == "dry cargo" or vessel_type == "container":
            EEDI_req = 10 + 13 * math.exp( DWT / (-470)) + 8 * math.exp( DWT / (-4500))      
        elif vessel_type == "pushed convoy":
            EEDI_req = 10 + 13 * math.exp( DWT / (-470)) + 8 * math.exp( DWT / (-4500))
        # note that the required_EEDI calculations for pushed convoys and self-propelled ships are the same in deep calm water!!

    elif 3.5 <= h <= 7.5:
        if vessel_type == "dry cargo" or vessel_type == "container":
            EEDI_req = (21 + 0.7 * U_c + 0.28 * U_c**2) + (11 + 0.78 * U_c - 0.46 * U_c**2 + 0.154 * U_c**3)* math.exp(DWT / (-800))       
        elif vessel_type == "pushed convoy":
            EEDI_req = (18 - 2.5 * U_c + 0.75 * U_c**2) + (8 + 0.25 * U_c + 0.375 * U_c**2) * math.exp(DWT / (-3100))
    else:
            print ("out of the water depth range or wrong vessel type input")        
        
    
    return EEDI_req

In [9]:
EEDI_req = calculate_required_EEDI(h = 4, U_c = 2,  vessel_type = "pushed convoy", DWT =1000)
EEDI_req

23.24277519974214

In [10]:
EEDI_req = calculate_required_EEDI(h = 4, U_c = 2,  vessel_type = "container", DWT =250)
EEDI_req

32.26426999717026

In [11]:
EEDI_req = calculate_required_EEDI(h = 14, U_c = 2,  vessel_type = "container", DWT =18000)
EEDI_req

10.146525111109874

In [12]:
# to do input table

### Application case on RhineARA corrior
take U_c = 4 km/h as the general current speed, take h = 10 m representing deep water and h = 4 m representing shallow water

####  input value

In [13]:
h = [4, 10]  
U_c = [4]
vessel_type = ["dry cargo","container","pushed convoy"]
DWT = [11200, 5520, 5320, 5500, 5686, 3043, 5920, 6228, 3300, 3043, 8134, 2973, 1680, 3257, 3043, 2211, 2075, 2908, 985,1522,1718 ]
B = [15, 11.4, 11.45, 17, 14.2, 9.5, 8.2]
V_g = [12.4, 10, 10.8, 10.1, 9.9, 8.9, 9.5, 11.4, 10.3, 13.2, 10.4, 9.6, 10.5]

#### prepare input matrix for calculation

In [14]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product(vessel_type, h, U_c, DWT, B, V_g))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"vessel_type": item[0], "h": item[1], "U_c": item[2], "DWT": item[3], "B": item[4], "V_g": item[5]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.head()

,vessel_type,h,U_c,DWT,B,V_g
0,dry cargo,4,4,11200,15.0,12.4
1,dry cargo,4,4,11200,15.0,10.0
2,dry cargo,4,4,11200,15.0,10.8
3,dry cargo,4,4,11200,15.0,10.1
4,dry cargo,4,4,11200,15.0,9.9


In [15]:

results = []

for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    vessel_type = row['vessel_type']
    h = row['h']
    U_c = row['U_c']
    DWT = row['DWT']
    B = row['B']    
    V_g = row['V_g']


    P_D = get_P_D( B, h, vessel_type, DWT)
    EEDI_attained = calculate_attained_EEDI( B, h, vessel_type, DWT, V_g)
    EEDI_req = calculate_required_EEDI(h, U_c, vessel_type, DWT)
        
    result = {}
    result.update(row)
    result['P_D'] = P_D
    result['EEDI_req (g CO2 / ton-km)'] = EEDI_req    
    result['EEDI_attained (g CO2 / ton-km)'] = EEDI_attained

    results.append(result)

11466it [00:00, 12457.51it/s]


In [16]:
plot_df = pd.DataFrame(results)

plot_df.head(20)

,vessel_type,h,U_c,DWT,B,V_g,P_D,EEDI_req (g CO2 / ton-km),EEDI_attained (g CO2 / ton-km)
0,dry cargo,4,4,11200,15.0,12.4,2957.546046,28.280014,15.020279
1,dry cargo,4,4,11200,15.0,10.0,2957.546046,28.280014,18.625146
2,dry cargo,4,4,11200,15.0,10.8,2957.546046,28.280014,17.245506
3,dry cargo,4,4,11200,15.0,10.1,2957.546046,28.280014,18.440739
4,dry cargo,4,4,11200,15.0,9.9,2957.546046,28.280014,18.813279
5,dry cargo,4,4,11200,15.0,8.9,2957.546046,28.280014,20.927131
6,dry cargo,4,4,11200,15.0,9.5,2957.546046,28.280014,19.605417
7,dry cargo,4,4,11200,15.0,11.4,2957.546046,28.280014,16.337848
8,dry cargo,4,4,11200,15.0,10.3,2957.546046,28.280014,18.082666
9,dry cargo,4,4,11200,15.0,13.2,2957.546046,28.280014,14.109959


In [17]:
# resistance for water depth h_0 = 7.5 m, 1990
Rotterdam_Duisburg_PushB4 = plot_df.query('vessel_type =="dry cargo" & DWT == 11200 & h == 4 & B == 15  & V_g == 12.4')
Rotterdam_Duisburg_PushB4 



,vessel_type,h,U_c,DWT,B,V_g,P_D,EEDI_req (g CO2 / ton-km),EEDI_attained (g CO2 / ton-km)
0,dry cargo,4,4,11200,15.0,12.4,2957.546046,28.280014,15.020279
